In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.9 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split


In [4]:
# Load the IMDB dataset from CSV
df = pd.read_csv('/content/drive/MyDrive/IMDB_Dataset.csv', encoding='utf-8',on_bad_lines='skip')


In [5]:
test_df = pd.read_csv('/content/drive/MyDrive/movie-review-dataset.csv', on_bad_lines='skip')

In [6]:
print(len(test_df))

609


In [7]:
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})

print(df.head(20))

                                               review  sentiment
0   One of the other reviewers has mentioned that ...          1
1   A wonderful little production. <br /><br />The...          1
2   I thought this was a wonderful way to spend ti...          1
3   Basically there's a family where a little boy ...          0
4   Petter Mattei's "Love in the Time of Money" is...          1
5   Probably my all-time favorite movie, a story o...          1
6   I sure would like to see a resurrection of a u...          1
7   This show was an amazing, fresh & innovative i...          0
8   Encouraged by the positive comments about this...          0
9   If you like original gut wrenching laughter yo...          1
10  Phil the Alien is one of those quirky films wh...          0
11  I saw this movie when I was about 12 when it c...          0
12  So im not a big fan of Boll's work but then ag...          0
13  The cast played Shakespeare.<br /><br />Shakes...          0
14  This a fantastic movi

In [8]:
# Preprocess the text column
import re
import pandas as pd
import numpy as np
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove html tags
    text = re.sub('<[^<]+?>', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

df['review'] = df['review'].apply(preprocess_text)

In [9]:
df1 = df[df['review'].str.split().str.len() < 100]

print(df1.head(10))


                                               review  sentiment
9   if you like original gut wrenching laughter yo...          1
10  phil the alien is one of those quirky films wh...          0
14  this a fantastic movie of three prisoners who ...          1
22  what an absolutely stunning movie if you have ...          1
36  the plot is about the death of little children...          0
46  protocol is an implausible movie whose only sa...          0
49  average and surprisingly tame fulci giallo whi...          0
53  i cannot believe i enjoyed this as much as i d...          1
62  so lets beginthe movie itself is as original a...          1
63  besides being boring the scenes were oppressiv...          0


In [10]:
print(len(df1))

6510


In [11]:
# Define tokenizer
import transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



In [12]:
# Tokenize train dataset
import torch
# Tokenize train dataset
train_encodings = tokenizer(list(df1['review']), padding=True, truncation=True, max_length=100)
X_train = train_encodings['input_ids']
y_train = np.array(list(df1['sentiment']))
print(len(X_train))
print(len(y_train))



6510
6510


In [13]:
# Tokenize test dataset
test_encodings = tokenizer(list(test_df['review']), padding=True, truncation=True, max_length=100)
X_val = test_encodings['input_ids']
y_val = np.array(list(test_df['sentiment']))

In [14]:
from transformers import TFBertForSequenceClassification

# Load saved model
model_path = '/content/drive/MyDrive/Model/Model-PreTrained'
model = TFBertForSequenceClassification.from_pretrained(model_path)
# Load the BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at /content/drive/MyDrive/Model/Model-PreTrained were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Model/Model-PreTrained.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [15]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [16]:
# Train the model
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_train, "attention_mask": np.ones_like(X_train)},
    y_train
))
train_dataset = train_dataset.shuffle(len(X_train)).batch(32)

In [17]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": X_val, "attention_mask": np.ones_like(X_val)},
    y_val
))
val_dataset = val_dataset.batch(32)

In [ ]:
# Evaluate the model on the validation dataset
results = model.evaluate(val_dataset)

# Print the accuracy
print(f"Accuracy: {results[1]*100:.2f}%")


20/20 [==============================] - 59s 2s/step - loss: 0.1477 - accuracy: 0.9639
Accuracy: 96.39%


In [18]:
!pip install --upgrade textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 69.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/9

In [19]:
!pip show textattack

Name: textattack
Version: 0.3.8
Summary: A library for generating text adversarial examples
Home-page: https://github.com/QData/textattack
Author: QData Lab at the University of Virginia
Author-email: jm8wx@virginia.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: bert-score, click, datasets, editdistance, filelock, flair, jieba, language-tool-python, lemminflect, lru-dict, more-itertools, nltk, num2words, numpy, OpenHowNet, pandas, pinyin, pycld2, PySocks, scipy, terminaltables, torch, tqdm, transformers, word2number
Required-by: 


In [20]:
import textattack.attack_recipes as attack_recipes

# Get a list of all the methods and attributes in attack_recipes.
methods = [method for method in dir(attack_recipes) if callable(getattr(attack_recipes, method))]

# Print the list of methods.
print(methods)



textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['A2TYoo2021', 'AttackRecipe', 'BAEGarg2019', 'BERTAttackLi2020', 'CLARE2020', 'CheckList2020', 'DeepWordBugGao2018', 'FasterGeneticAlgorithmJia2019', 'FrenchRecipe', 'GeneticAlgorithmAlzantot2018', 'HotFlipEbrahimi2017', 'IGAWang2019', 'InputReductionFeng2018', 'Kuleshov2017', 'MorpheusTan2020', 'PSOZang2020', 'PWWSRen2019', 'Pruthi2019', 'Seq2SickCheng2018BlackBox', 'SpanishRecipe', 'TextBuggerLi2018', 'TextFoolerJin2019']


In [21]:
import torch
import transformers
import textattack
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_recipes import TextBuggerLi2018

from transformers import BertForSequenceClassification, BertTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
import numpy as np


In [22]:

class MyModelWrapper(ModelWrapper):
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __call__(self, text_input_list):
        input_ids = []
        token_type_ids = []
        attention_mask = []
        for text_input in text_input_list:
            input_dict = self.tokenizer.encode_plus(text_input, add_special_tokens=True, max_length=512, pad_to_max_length=True)
            input_ids.append(input_dict['input_ids'])
            token_type_ids.append(input_dict['token_type_ids'])
            attention_mask.append(input_dict['attention_mask'])
            
        input_ids = np.array(input_ids)
        token_type_ids = np.array(token_type_ids)
        attention_mask = np.array(attention_mask)
        outputs = self.model([input_ids, token_type_ids, attention_mask], training=False)
        logits = outputs[0]
        return logits.numpy()


In [23]:
model_wrapper = MyModelWrapper(model, tokenizer)

In [24]:
# Initialize the TextFooler attack recipe
attack = TextBuggerLi2018.build(model_wrapper)

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:38<00:00, 12.5MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp6vuh5tbm.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [25]:
# test_df.columns = ['review', 'sentiment']
# test_df.to_csv('test1.csv', index=False)
texts = np.array(test_df['review'])
labels = np.array(test_df['sentiment'])

In [ ]:
from textattack.shared import AttackedText
text = texts[0]
label = labels[0]

results_iterable = attack.attack(text, ground_truth_output=int(label))
adv_text = results_iterable.perturbed_text()

print(adv_text)

In [26]:
adv_texts = []
for text, label in zip(texts, labels):
    results_iterable = attack.attack(text, ground_truth_output=int(label))
    if hasattr(results_iterable, "__iter__"):
        for result in results_iterable:
            adv_text = results_iterable.perturbed_text()
            adv_texts.append(adv_text)
            print(f"Adversarial example: {adv_text}")
    else:
        adv_text = results_iterable.perturbed_text()
        adv_texts.append(adv_text)
        print(f"Adversarial example: {adv_text}")



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Adversarial example: This movie was greɑt!
Adversarial example: Thе acting was dreadful
Adversarial example: I loved this films
Adversarial example: It was a waste of tme
Adversarial example: The plot was foreseeable
Adversarial example: This is a mus𝚝-watch movie
Adversarial example: I woulԁ not recommend this movie
Adversarial example: The cinematography was astonishing
Adversarial example: The script was frail
Adversarial example: The ending was disappoin𝚝ing
Adversarial example: T his is the best movie ever!
Adversarial example: I regretful watching this film
Adversarial example: The character were well-developed
Adversarial example: It was too sloԝ-paced
Adversarial example: I was on the edge of my seat the whole time
Adversarial example: The special effects were amazіng
Adversarial example: I was bores throughout the entire movie
Adversarial example: The soundtrack was beau𝚝iful
Adversarial example: Thе dialogues was cheesy
Adversarial example: I would watch this movie again and 

In [27]:
import csv
csv_file_path = '/content/drive/MyDrive/Model/TextBuggerLi2018_adv_examples.csv'
# Write the adversarial examples to the CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['review']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for example in adv_texts:
        writer.writerow({'review': example})

In [28]:
# Tokenize the adversarial texts
adv_inputs = tokenizer(list(adv_texts), padding=True, truncation=True, max_length=128, return_tensors='tf')

# Evaluate accuracy using the adversarial examples
adv_preds = model_wrapper.model(adv_inputs).logits.numpy().argmax(axis=1)
adv_acc = (adv_preds == labels).mean()

print(f"Adversarial accuracy: {adv_acc*100:.4f}%")

Adversarial accuracy: 79.9672%
